In [305]:
import requests
import json
import pandas as pd
import csv
import json

Подробное описание парсера Вайлдберриз можно почитать на сайте:
https://happypython.ru/2022/07/21/парсер-wildberries/
Ссылка на статью ВКонтакте: https://vk.com/@happython-parser-wildberries
По всем возникшим вопросам, можете писать в группу https://vk.com/happython
парсер wildberries по каталогам 2022, обновлен 19.10.2022 - на данное число работает исправно
---

Получение списка всех каталогов wildberries. Функция **get_catalogs_wb**. Получаем в json формате и сохраняем в файл **wb_catalogs_data.json**.

In [135]:
def get_catalogs_wb():
    """получение каталога вб"""
    url = 'https://www.wildberries.ru/webapi/menu/main-menu-ru-ru.json'
    headers = {'Accept': "*/*", 'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    response = requests.get(url, headers=headers)
    data = response.json()
  #  print(data)
    with open('wb_catalogs_data.json', 'w', encoding='UTF-8') as file:
        json.dump(data, file, indent=2, ensure_ascii=False)
        print(f'Данные сохранены в wb_catalogs_data_sample.json')
    data_list = []
    for d in data:
        try:
            for child in d['childs']:
                try:
                    category_name = child['name']
                    category_url = child['url']
                    shard = child['shard']
                    query = child['query']
                    data_list.append({
                        'category_name': category_name,
                        'category_url': category_url,
                        'shard': shard,
                        'query': query})
                except:
                    continue
                try:
                    for sub_child in child['childs']:
                        category_name = sub_child['name']
                        category_url = sub_child['url']
                        shard = sub_child['shard']
                        query = sub_child['query']
                        data_list.append({
                            'category_name': category_name,
                            'category_url': category_url,
                            'shard': shard,
                            'query': query})
                except:
                    # print(f'не имеет дочерних каталогов *{i["name"]}*')
                    continue
        except:
            # print(f'не имеет дочерних каталогов *{d["name"]}*')
            continue
    return data_list

Поиск пользовательской категории в списке каталогов. Функция **search_category_in_catalog**

In [ ]:
def search_category_in_catalog(url, catalog_list):
    """пишем проверку пользовательской ссылки на наличии в каталоге"""
    try:
        for catalog in catalog_list:
            if catalog['category_url'] == url.split('https://www.wildberries.ru')[-1]:
                print(f'найдено совпадение: {catalog["category_name"]}')
                name_category = catalog['category_name']
                shard = catalog['shard']
                query = catalog['query']
                return name_category, shard, query
            else:
                # print('нет совпадения')
                pass
    except:
        print('Данный раздел не найден!')

Извлечение данных из json файла с информацией о товарах. Функция **get_data_from_json**

In [136]:
def get_data_from_json(json_file):
    """извлекаем из json интересующие нас данные"""
    data_list = []
    for data in json_file['data']['products']:
        try:
            price = int(data["priceU"] / 100)
        except:
            price = 0
        data_list.append({
            'Наименование': data['name'],
            'id': data['id'],
            'Скидка': data['sale'],
            'Цена': price,
            'Цена со скидкой': int(data["salePriceU"] / 100),
            'Бренд': data['brand'],
            'id бренда': int(data['brandId']),
            'feedbacks': data['feedbacks'],
            'rating': data['rating'],
            'Ссылка': f'https://www.wildberries.ru/catalog/{data["id"]}/detail.aspx?targetUrl=BP'
        })
    return data_list

Постраничный сбор данных. Функция **get_content**

In [137]:
def get_content(shard, query, low_price=None, top_price=None):
    # вставляем ценовые рамки для уменьшения выдачи, вилбериес отдает только 100 страниц
    headers = {'Accept': "*/*", 'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    data_list = []
    for page in range(1, 29):
        print(f'Сбор позиций со страницы {page} из 100')
        url = f'https://catalog.wb.ru/catalog/{shard}/catalog?appType=1&curr=rub&dest=-1075831,-77677,-398551,12358499' \
              f'&locale=ru&page={page}&priceU={low_price * 100};{top_price * 100}' \
              f'&reg=0&regions=64,83,4,38,80,33,70,82,86,30,69,1,48,22,66,31,40&sort=popular&spp=0&{query}'
        r = requests.get(url, headers=headers)
        data = r.json()
        print(f'Добавлено позиций: {len(get_data_from_json(data))}')
        if len(get_data_from_json(data)) > 0:
            data_list.extend(get_data_from_json(data))
        else:
            print(f'Сбор данных завершен.')
            break
       # print(data_list)
    return data_list

Сохранение данных в эксель файл при помощи библиотеки pandas. Функция **save_excel**

In [ ]:
def save_excel(data, filename):
    """сохранение результата в csv файл"""
    df = pd.DataFrame(data)
    df.to_csv(f'{filename}.csv')
    print(f'Все сохранено в {filename}.csv')

Основная функция выполняющая все выше перечисленные. Функция **parser**

In [138]:
def parser(url, low_price, top_price):
    # получаем список каталогов
    catalog_list = get_catalogs_wb()
    try:
        # поиск введенной категории в общем каталоге
        name_category, shard, query = search_category_in_catalog(url=url, catalog_list=catalog_list)
        # сбор данных в найденном каталоге
        data_list = get_content(shard=shard, query=query, low_price=low_price, top_price=top_price)
        # сохранение найденных данных
        save_excel(data_list, f'{name_category}_from_{low_price}_to_{top_price}')
    except TypeError:
        print('Ошибка! Возможно не верно указан раздел. Удалите все доп фильтры с ссылки')
    except PermissionError:
        print('Ошибка! Вы забыли закрыть созданный ранее excel файл. Закройте и повторите попытку')


if __name__ == '__main__':
    """ссылку на каталог или подкаталог, указывать без фильтров (без ценовых, сортировки и тд.)"""
    # url = input('Введите ссылку на категорию для сбора: ')
    # low_price = int(input('Введите минимальную сумму товара: '))
    # top_price = int(input('Введите максимульную сумму товара: '))

    """ собераем данные на товар с раздела телефоны и гаджеты в ценовой категории от 1тыс, до 200тыс"""
    url = 'https://www.wildberries.ru/catalog/elektronika/telefony-i-gadzhety/mobilnye-telefony'
    low_price = 1000
    top_price = 200000

    parser(url, low_price, top_price)

Данные сохранены в wb_catalogs_data_sample.json
найдено совпадение: Мобильные телефоны
Сбор позиций со страницы 1 из 100
Добавлено позиций: 100
Сбор позиций со страницы 2 из 100
Добавлено позиций: 100
Сбор позиций со страницы 3 из 100
Добавлено позиций: 100
Сбор позиций со страницы 4 из 100
Добавлено позиций: 100
Сбор позиций со страницы 5 из 100
Добавлено позиций: 100
Сбор позиций со страницы 6 из 100
Добавлено позиций: 100
Сбор позиций со страницы 7 из 100
Добавлено позиций: 100
Сбор позиций со страницы 8 из 100
Добавлено позиций: 100
Сбор позиций со страницы 9 из 100
Добавлено позиций: 100
Сбор позиций со страницы 10 из 100
Добавлено позиций: 100
Сбор позиций со страницы 11 из 100
Добавлено позиций: 100
Сбор позиций со страницы 12 из 100
Добавлено позиций: 100
Сбор позиций со страницы 13 из 100
Добавлено позиций: 100
Сбор позиций со страницы 14 из 100
Добавлено позиций: 100
Сбор позиций со страницы 15 из 100
Добавлено позиций: 100
Сбор позиций со страницы 16 из 100
Добавлено позиций

In [159]:
data = pd.read_csv('Мобильные телефоны_from_1000_to_200000.csv', sep=',')
data.head()

,Unnamed: 0,Наименование,id,Скидка,Цена,Цена со скидкой,Бренд,id бренда,feedbacks,rating,Ссылка
0,0,Игровой набор Рации детские Игровые телефоны р...,39064854,71,9800,2842,S&H Smart Kids,452254,102,5,https://www.wildberries.ru/catalog/39064854/de...
1,1,Мобильный телефон 2810 BOOM XL,7118843,45,2958,1626,BQ,23870,404,5,https://www.wildberries.ru/catalog/7118843/det...
2,2,Игровой набор Рации детские Игровые телефоны р...,78143114,69,9200,2852,S&H Smart Kids,452254,102,5,https://www.wildberries.ru/catalog/78143114/de...
3,3,"Мобильный телефон 225 4G DS 2,4 QVGA 64+128МБ ...",17145089,34,3890,2567,Nokia,16111,90,4,https://www.wildberries.ru/catalog/17145089/de...
4,4,Мобильный телефон 2810 BOOM XL,7118846,45,2958,1626,BQ,23870,404,5,https://www.wildberries.ru/catalog/7118846/det...
